In [190]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("train-images-idx3-ubyte", "train-labels-idx1-ubyte",
        "mnist_train.csv", 60000)
convert("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte",
        "mnist_test.csv", 10000)

In [191]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [192]:
df_orig_train = pd.read_csv('mnist_train.csv')
df_orig_test = pd.read_csv('mnist_test.csv')

In [193]:
df_orig_train.rename(columns={'5':'label'}, inplace=True)
df_orig_test.rename(columns={'7':'label'}, inplace=True)

In [194]:
df_orig_train.head()

,label,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [195]:
df_orig_test.head()

,label,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [196]:
# Writing final version of files
df_orig_train.to_csv('mnist_train_final.csv', index=False)
df_orig_test.to_csv('mnist_test_final.csv', index=False)

In [197]:
df= pd.read_csv("mnist_train_final.csv")
df2=pd.read_csv("mnist_test_final.csv")


In [198]:
df=np.array(df)
df2=np.array(df2)

In [199]:
#set aside dev data for cross validation
m, n = df.shape
r, s=df2.shape
#n is no. of columns + 1
np.random.shuffle(df) # shuffle before splitting into dev and training sets

df2=df2.T
y_test = df2[0]
x_test = df2[1:s]
x_test = x_test / 255.


df=df.T
y_train = df[0]
x_train = df[1:n]
x_train = x_train / 255.
_,m_train = x_train.shape


In [200]:
def init_params():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return w1, b1, w2, b2

In [201]:
def ReLU(z):
    return np.maximum(z,0)

def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def forward_prop(w1, b1, w2, b2,x):
    z1=w1.dot(x)+b1
    a1=ReLU(z1)
    z2=w2.dot(a1)+b2
    a2=softmax(z2)
    return z1,a1,z2,a2

    
 

In [202]:
#y is our labels
def onehot(y):
    onehot_y=np.zeros((y.size,y.max()+1))
    #1,10
    onehot_y[np.arange(y.size),y]=1
    onehot_y= onehot_y.T
    return onehot_y

    #to make each row a class and each column a example 
    #10,1 for each pass

def ReLU_deriv(z):
    return (z>0)
    #if z>0, we return 1, else 0 (according to graph)

def back_prop(z1,a1,z2,a2,w1,w2,x,y):
    m=y.size
    onehot_y=onehot(y)
    dz2=a2-onehot_y
    dw2=1/m*dz2.dot(a1.T)
    db2=1/m*np.sum(dz2)
    dz1=w2.T.dot(dz2)*ReLU_deriv(z1)
    dw1=1/m*dz1.dot(x.T)
    db1=1/m*np.sum(dz1)
    return dw1,db1,dw2,db2

def update(dw1,db1,dw2,db2,w1,w2,b1,b2,alpha):
    w1=w1-alpha*dw1
    b1=b1-alpha*db1
    w2=w2-alpha*dw2
    b2=b2-alpha*db2
    return w1,b1,w2,b2


In [203]:
def get_predictions(a2):
    return np.argmax(a2,0)

def get_accuracy(predictions,y):
    print(predictions, y)
    return (np.sum(predictions==y)/y.size)


In [204]:
def gradient_descent(x,y,iterations,alpha):
    w1,b1,w2,b2=init_params()
    for i in range(iterations):
        z1,a1,z2,a2=forward_prop(w1,b1,w2,b2,x)
        dw1,db1,dw2,db2=back_prop(z1,a1,z2,a2,w1,w2,x,y)
        w1,b1,w2,b2=update(dw1,db1,dw2,db2,w1,w2,b1,b2,alpha)
        if i%100==0:
            print("iteration: ",i)
            print("Accuracy: ", get_accuracy(get_predictions(a2),y))
    
    return w1,b1,w2,b2


In [205]:
w1,b1,w2,b2=gradient_descent(x_train,y_train,1500,0.5)

iteration:  0
[4 4 8 ... 4 4 4] [0 8 1 ... 9 5 1]
Accuracy:  0.10738512308538475
iteration:  100
[0 8 1 ... 4 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.8151135852264204
iteration:  200
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.8673311221853698
iteration:  300
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.8847814130235504
iteration:  400
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.8944982416373606
iteration:  500
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9006983449724162
iteration:  600
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9052484208070134
iteration:  700
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9097818296971616
iteration:  800
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9123152052534209
iteration:  900
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9141152352539209
iteration:  1000
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9182819713661894
iteration:  1100
[0 8 1 ... 9 5 1] [0 8 1 ... 9 5 1]
Accuracy:  0.9206153435890598
iteration:  120

In [206]:
z1,a1,z2,a2=forward_prop(w1,b1,w2,b2,x_test)
y_pred= get_predictions(a2)
get_accuracy(y_pred,y_test)



[2 1 0 ... 4 5 6] [2 1 0 ... 4 5 6]


0.9253925392539254